In [ ]:
# Import essential libraries and frameworks

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
import random
import matplotlib.image as mpimg

In [ ]:
# Define train and test datasets path

train_path = "intel-image-classification/seg_train/seg_train"
test_path = "intel-image-classification/seg_test/seg_test"

# Following are the image classes in the dataset.
print("Classes:", os.listdir(train_path))

In [ ]:
# Display random image from training set

folders = os.listdir(train_path)
folder = random.choice(folders)
image_file = random.choice(os.listdir(os.path.join(train_path, folder)))
image_path = os.path.join(train_path, folder, image_file)

img = mpimg.imread(image_path)
plt.imshow(img)
plt.title(f"Class: {folder}")
plt.axis('off')
plt.show()

In [ ]:
# Preprocessing of image with ImageDataGenerator

IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# Data Augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

val_gen = val_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
# Get one batch from the generator
images, labels = next(train_gen)

# Print shape
print("Images shape:", images.shape)  # e.g., (32, 150, 150, 3)
print("Labels shape:", labels.shape)  # e.g., (32, 6)
print("Labels shape:", labels) 

In [ ]:
# Get mapping of class index to class name
index_to_class = {v: k for k, v in train_gen.class_indices.items()}

# Show first 5 images and their labels
for i in range(5):
    plt.imshow(images[i])
    plt.axis('off')

    # Get one-hot label and class index
    one_hot = labels[i]
    class_index = np.argmax(one_hot)
    class_name = index_to_class[class_index]

    plt.title(f"Label: {class_name}\nOne-hot: {one_hot}")
    plt.show()


In [ ]:
# Model Architecture

model = models.Sequential([
    layers.Input((150, 150, 3)),

    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(192, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(6, activation='softmax')  # Output layer for 6 classes
])

In [ ]:
# Callbacks

early_stop = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)

lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

In [ ]:
# Model Training

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=40,
    callbacks=[early_stop, lr_scheduler]
)

In [ ]:
model.summary()

In [ ]:
# Accuracy & Loss Curves

plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.legend()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_gen)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Step 1: Get predictions
test_gen.reset()  # Important to reset before predicting
preds = model.predict(test_gen, steps=test_gen.samples // test_gen.batch_size + 1, verbose=1)

# Step 2: Convert predictions to class indices
y_pred = np.argmax(preds, axis=1)

# Step 3: True labels
y_true = test_gen.classes  # Already integer-encoded

# Step 4: Classification report
print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))

# Step 5: Confusion Matrix

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=test_gen.class_indices.keys(), yticklabels=test_gen.class_indices.keys(), cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# You can pass test images to test prediction

class_labels = list(train_gen.class_indices.keys())

# Load and preprocess the image
img = image.load_img('any-image-path.jpeg', target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # shape = (1, 150, 150, 3)

# Predict
prediction = model.predict(img_array)
predicted_class = class_labels[np.argmax(prediction)]

print(f"Predicted Class: {predicted_class}")
